# Import DataFrame and Prep Model

In [0]:
#Arrays and Dataframe
import numpy as np
import pandas as pd

#SQL
from sqlalchemy import create_engine

#Visualization
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

#Data Exploration
from scipy import stats

#Data Modeling
from sklearn import linear_model
from sklearn.svm import LinearSVC
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from statsmodels.tools.tools import add_constant

from statsmodels.tsa.stattools import acf

from scipy.stats import jarque_bera
from scipy.stats import normaltest

In [3]:
#Make SQL Queary
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [0]:
#Drop columns that have a large number of null values (over 30%)
df1 = df.drop(columns = ['alley', 'fireplacequ', 'poolqc', 'fence', 'miscfeature'])

In [0]:
#make a copy of df1
df2 = df1.copy()

#interpolate based on the year
year_list = list(df1['yearbuilt'].unique())
for year in year_list:
    df2.loc[df1['yearbuilt']==year] = df2.loc[df1['yearbuilt']==year].interpolate()

# drop missing values that are remaining
df2.dropna(inplace = True)

In [0]:
#update data types
non_numeric = df1.select_dtypes(include = 'object')
numeric = df1.select_dtypes(include = ['float64', 'int64'])

In [0]:
#pick top 5 numeric:
df3 = df2.loc[:, ['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf', 'saleprice']]

In [0]:
#Choose Central Air and External Quality as the features I'll use for the model
df4 = df2.loc[:, ['exterqual', 'centralair']]

#One Hot encoding of categorical variables
df5 = pd.get_dummies(df4, columns = ['exterqual', 'centralair'])

In [0]:
#Combine Dataframes
df3
df5
model_df = pd.concat([df3, df5], axis = 1)

# Create Model


## Run 1st Model

In [0]:
#Separate target variable from rest of data
X = model_df.drop(columns = 'saleprice')
y = model_df['saleprice']

#split train_test
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [11]:
X = sm.add_constant(X)

results = sm.OLS(y, X).fit()

results.summary()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.775
Model:                            OLS   Adj. R-squared:                  0.773
Method:                 Least Squares   F-statistic:                     500.0
Date:                Tue, 05 Nov 2019   Prob (F-statistic):               0.00
Time:                        20:18:06   Log-Likelihood:                -15757.
No. Observations:                1318   AIC:                         3.153e+04
Df Residuals:                    1308   BIC:                         3.159e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -4.127e+04   4923.196     -8.383      0.000   -5.09e+04   -3.16e+04
overallqual   1.884e+04   1335.192     14.107      0.000    1.62e+04    2.15e+04
grlivarea       47.6567      2.612     18.249      0.000      42.533      52.780
garagecars    1.645e+04   3147.805      5.227      0.000    1.03e+04    2.26e+04
garagearea      13.4964     10.494      1.286      0.199      -7.091      34.084
totalbsmtsf     26.2237      3.274      8.011      0.000      19.802      32.646
exterqual_Ex  3.735e+04   6542.041      5.709      0.000    2.45e+04    5.02e+04
exterqual_Fa -3.957e+04   1.15e+04     -3.435      0.001   -6.22e+04    -1.7e+04
exterqual_Gd -1.261e+04   4084.879     -3.086      0.002   -2.06e+04   -4591.762
exterqual_TA -2.645e+04   3568.598     -7.411      0.000   -3.34e+04   -1.94e+04
centralair_N -3.273e+04   3714.024     -8.812      0.000      -4e+04   -2.54e+04
centralair_Y -8544.2175   3417.795     -2.500      0.013   -1.52e+04   -1839.257
==============================================================================
Omnibus:                      603.343   Durbin-Watson:                   1.972
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            68036.538
Skew:                          -1.125   Prob(JB):                         0.00
Kurtosis:                      38.126   Cond. No.                     5.96e+19
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.55e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

This model has a high F statistic (500). The R-sqared and R-adjusted are around 77%, which means that 77% of the variance can be explained by the current features. The only feauture that seems to be statistically insignificant is garagearea. This will be removed and the model retested. 

## Run Second Model

In [0]:
#Separate target variable from rest of data
X1 = model_df.drop(columns = ['saleprice', 'garagearea'])
y = model_df['saleprice']

In [13]:
X1 = sm.add_constant(X1)

results1 = sm.OLS(y, X1).fit()

results1.summary()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.775
Model:                            OLS   Adj. R-squared:                  0.773
Method:                 Least Squares   F-statistic:                     562.0
Date:                Tue, 05 Nov 2019   Prob (F-statistic):               0.00
Time:                        20:20:10   Log-Likelihood:                -15758.
No. Observations:                1318   AIC:                         3.153e+04
Df Residuals:                    1309   BIC:                         3.158e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -4.095e+04   4918.147     -8.327      0.000   -5.06e+04   -3.13e+04
overallqual   1.876e+04   1334.287     14.061      0.000    1.61e+04    2.14e+04
grlivarea       47.9542      2.602     18.430      0.000      42.850      53.059
garagecars    1.941e+04   2147.534      9.040      0.000    1.52e+04    2.36e+04
totalbsmtsf     27.1598      3.192      8.507      0.000      20.897      33.423
exterqual_Ex  3.745e+04   6543.184      5.724      0.000    2.46e+04    5.03e+04
exterqual_Fa -3.856e+04   1.15e+04     -3.354      0.001   -6.11e+04    -1.6e+04
exterqual_Gd   -1.3e+04   4074.614     -3.189      0.001    -2.1e+04   -5002.082
exterqual_TA -2.685e+04   3556.073     -7.549      0.000   -3.38e+04   -1.99e+04
centralair_N -3.267e+04   3714.701     -8.795      0.000      -4e+04   -2.54e+04
centralair_Y -8280.1366   3412.473     -2.426      0.015    -1.5e+04   -1585.622
==============================================================================
Omnibus:                      585.883   Durbin-Watson:                   1.972
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            63696.739
Skew:                          -1.071   Prob(JB):                         0.00
Kurtosis:                      36.990   Cond. No.                     6.69e+19
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.15e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

The F statistic improved with the removal of garagearea. R-squared and r-adjusted remained the same. AIC remained the same while BIC had a minor improvement. 

## Run 3rd Model

In [0]:
#Separate target variable from rest of data - also remove central air
X2 = model_df.drop(columns = ['saleprice', 'garagearea', 'centralair_N', 'centralair_Y'])
y = model_df['saleprice']

In [15]:
X2 = sm.add_constant(X2)

results2 = sm.OLS(y, X2).fit()

results2.summary()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.771
Model:                            OLS   Adj. R-squared:                  0.769
Method:                 Least Squares   F-statistic:                     628.9
Date:                Tue, 05 Nov 2019   Prob (F-statistic):               0.00
Time:                        20:25:00   Log-Likelihood:                -15769.
No. Observations:                1318   AIC:                         3.155e+04
Df Residuals:                    1310   BIC:                         3.159e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -5.458e+04   6916.130     -7.891      0.000   -6.81e+04    -4.1e+04
overallqual    1.95e+04   1335.707     14.602      0.000    1.69e+04    2.21e+04
grlivarea       46.6995      2.609     17.898      0.000      41.581      51.818
garagecars    1.968e+04   2164.181      9.091      0.000    1.54e+04    2.39e+04
totalbsmtsf     28.3624      3.208      8.841      0.000      22.069      34.656
exterqual_Ex  3.615e+04   6800.529      5.316      0.000    2.28e+04    4.95e+04
exterqual_Fa -5.058e+04   1.16e+04     -4.353      0.000   -7.34e+04   -2.78e+04
exterqual_Gd -1.307e+04   4082.332     -3.201      0.001   -2.11e+04   -5057.349
exterqual_TA -2.709e+04   3301.581     -8.205      0.000   -3.36e+04   -2.06e+04
==============================================================================
Omnibus:                      587.167   Durbin-Watson:                   1.972
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            60260.647
Skew:                          -1.092   Prob(JB):                         0.00
Kurtosis:                      36.054   Cond. No.                     2.33e+18
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 9.48e-28. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

This model improved only on f statistic but was not performing better in terms of r-squared and r-adjusted, or BIC/AIC.

I think the 2nd model is a satisfactory model because it explains 77% of the variance. If we get much higher than this, we run the risk of overfitting. 